In [0]:
# 模型错误类型分析脚本
# 该脚本会加载已训练好的 BiLSTM+CRF 模型，对测试集进行预测
import sys
from pathlib import Path
# 添加项目根目录到路径
project_root = Path(".").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
from collections import Counter, defaultdict
from copy import deepcopy
from typing import DefaultDict, Dict, List, Sequence, Tuple
try:
    from data import build_corpus
    from evaluating import Metrics
    from utils import extend_maps, load_model, prepocess_data_for_lstmcrf
except ImportError as e:
    print(f"导入错误: {e}")
    print("请确保在项目根目录运行此 Notebook，且相关模块可用")
DEFAULT_MODEL_PATH = Path(".").resolve() / "ckpts" / "bilstm_crf.pkl"

In [ ]:
def _ensure_best_model(model) -> None:
    """保证反序列化后的模型可以直接用于预测。"""
    if model.best_model is None:
        model.best_model = deepcopy(model.model)
    # 去除 PackedSequence 的警告，兼容 GPU/CPU
    if hasattr(model.best_model, "bilstm"):
        bilstm_block = getattr(model.best_model, "bilstm")
        if hasattr(bilstm_block, "bilstm"):
            bilstm_block.bilstm.flatten_parameters()

def _extract_entities_bmes(words: Sequence[str], tags: Sequence[str]) -> List[Dict[str, str]]:
    """
    将 BMES 标注转换为实体列表。
    """
    entities: List[Dict[str, str]] = []
    i = 0
    n = len(tags)
    
    while i < n:
        tag = tags[i]
        
        if tag.startswith("B-"):
            # 处理多字实体：B -> M* -> E
            entity_type = tag[2:]
            start = i
            j = i + 1
            
            # 寻找连续的M标签（同类型）
            while j < n and tags[j] == f"M-{entity_type}":
                j += 1
            
            # 检查是否以E标签结束（同类型）
            if j < n and tags[j] == f"E-{entity_type}":
                end = j + 1  # E标签包含在实体中
                entity_text = "".join(words[start:end])
                entities.append({
                    "type": entity_type, 
                    "text": entity_text, 
                    "span": f"{start}-{end}"
                })
                i = end
            else:
                # 不符合BMES规范，跳过这个B标签
                i += 1
                
        elif tag == "O":
            # 非实体，直接跳过
            i += 1
            
        else:
            # 遇到M或E但没有对应的B，跳过
            i += 1
            
    return entities

def _extract_entities_bmes_relaxed(words: Sequence[str], tags: Sequence[str]) -> List[Dict[str, str]]:
    """
    宽松的BMES实体提取。
    将连续的B/M/E（同类型）视为一个实体，即使中间有轻微的不规范。
    """
    entities: List[Dict[str, str]] = []
    i = 0
    n = len(tags)
    
    while i < n:
        tag = tags[i]
        
        if tag.startswith("B-"):
            entity_type = tag[2:]
            start = i
            end = i + 1
            
            # 寻找连续的同类标签
            j = i + 1
            while j < n:
                current_tag = tags[j]
                if (current_tag == f"M-{entity_type}" or 
                    current_tag == f"E-{entity_type}"):
                    end = j + 1
                    j += 1
                    # 如果遇到E标签，停止搜索
                    if current_tag.startswith("E-"):
                        break
                else:
                    break
            
            entity_text = "".join(words[start:end])
            entities.append({
                "type": entity_type,
                "text": entity_text, 
                "span": f"{start}-{end}"
            })
            i = end
        else:
            i += 1
            
    return entities

def _format_entities(entities: Sequence[Dict[str, str]]) -> str:
    """格式化实体列表用于显示"""
    if not entities:
        return "(无)"
    return " | ".join(f"{e['type']}:『{e['text']}』[{e['span']}]" for e in entities)

def _collect_errors(
    sentences: Sequence[Sequence[str]],
    golden_tags: Sequence[Sequence[str]],
    pred_tags: Sequence[Sequence[str]],
    max_examples_per_type: int = 3,
) -> Tuple[Counter, DefaultDict[str, List[Dict[str, str]]]]:
    """统计错误类型并返回典型案例。"""
    error_counter: Counter = Counter()
    error_examples: DefaultDict[str, List[Dict[str, str]]] = defaultdict(list)

    for words, gold_seq, pred_seq in zip(sentences, golden_tags, pred_tags):
        gold_entities = _extract_entities_bmes_relaxed(words, gold_seq)
        pred_entities = _extract_entities_bmes_relaxed(words, pred_seq)

        for idx, (gold, pred) in enumerate(zip(gold_seq, pred_seq)):
            if gold != pred:
                error_type = f"{gold}->{pred}"
                error_counter[error_type] += 1
                if len(error_examples[error_type]) < max_examples_per_type:
                    error_examples[error_type].append(
                        {
                            "sentence": "".join(words),
                            "words": words,
                            "gold_tags": gold_seq,
                            "pred_tags": pred_seq,
                            "gold_entities": gold_entities,
                            "pred_entities": pred_entities,
                        }
                    )
    return error_counter, error_examples

In [ ]:
def evaluate_model(model_path: Path = DEFAULT_MODEL_PATH, remove_o: bool = False) -> dict:
    """
    model_path : Path
        模型文件路径
    remove_o : bool
        是否在评估时忽略O标签
    Returns:dict: 包含评估结果的字典
    """
    print("读取数据...")
    try:
        _, _, word2id, tag2id = build_corpus("train")
        test_word_lists, test_tag_lists = build_corpus("test", make_vocab=False)
    except Exception as e:
        print(f"数据读取失败: {e}")
        return {}

    # 保留原始句子，便于展示错误案例
    display_word_lists = [list(seq) for seq in test_word_lists]

    crf_word2id, crf_tag2id = extend_maps(word2id, tag2id, for_crf=True)
    test_word_lists, test_tag_lists = prepocess_data_for_lstmcrf(
        test_word_lists, test_tag_lists, test=True
    )

    print(f"加载模型: {model_path}")
    try:
        bilstm_model = load_model(str(model_path))
        _ensure_best_model(bilstm_model)
    except Exception as e:
        print(f"模型加载失败: {e}")
        return {}

    print("开始预测...")
    pred_tag_lists, target_tag_lists = bilstm_model.test(
        test_word_lists, test_tag_lists, crf_word2id, crf_tag2id
    )

    print("\n基础评估指标：")
    metrics = Metrics(target_tag_lists, pred_tag_lists, remove_O=remove_o)
    metrics.report_scores()

    error_counter, error_examples = _collect_errors(
        display_word_lists, target_tag_lists, pred_tag_lists
    )
    
    return {
        'metrics': metrics,
        'error_counter': error_counter,
        'error_examples': error_examples,
        'pred_tags': pred_tag_lists,
        'true_tags': target_tag_lists,
        'sentences': display_word_lists
    }

In [ ]:
# 交互式错误分析函数
def interactive_error_analysis(results, error_type: str = None, max_examples: int = 5):
    """
    results : dict
        evaluate_model 函数的返回结果
    error_type : str, optional
        指定要查看的错误类型，如 'B-PER->O'
    max_examples : int
        最大显示案例数量
    """
    if not results:
        print("没有可用的结果")
        return
        
    error_counter = results['error_counter']
    error_examples = results['error_examples']
    
    # print("所有错误类型统计:")
    # for i, (err_type, count) in enumerate(error_counter.most_common()):
    #     print(f"{i+1:2d}. {err_type}: {count}")
    
    if error_type:
        if error_type in error_examples:
            examples = error_examples[error_type][:max_examples]
            print(f"\n详细查看错误类型: {error_type} (显示{len(examples)}个案例)")
            for j, case in enumerate(examples):
                print(f"\n--- 案例 {j+1} ---")
                print(f"句子: {case['sentence']}")
                print(f"字/词: {' '.join(case['words'])}")
                # print(f"Gold标签: {' '.join(case['gold_tags'])}")
                # print(f"Pred标签: {' '.join(case['pred_tags'])}")
                print(f"Gold实体: {_format_entities(case['gold_entities'])}")
                print(f"Pred实体: {_format_entities(case['pred_entities'])}")
                print("-" * 60)
        else:
            print(f"错误类型 '{error_type}' 不存在")

In [ ]:

# 实体对比分析函数
def compare_entities(results, sentence_index: int):
    """
    对比特定句子的实体识别结果
    
    Parameters:
    -----------
    results : dict
        evaluate_model 函数的返回结果
    sentence_index : int
        要分析的句子索引
    """
    if not results:
        print("没有可用的结果")
        return
        
    sentences = results['sentences']
    true_tags = results['true_tags']
    pred_tags = results['pred_tags']
    
    if sentence_index >= len(sentences):
        print(f"句子索引超出范围，最大索引为 {len(sentences)-1}")
        return
    
    words = sentences[sentence_index]
    gold_tags = true_tags[sentence_index]
    pred_tags_seq = pred_tags[sentence_index]
    
    gold_entities = _extract_entities_bmes_relaxed(words, gold_tags)
    pred_entities = _extract_entities_bmes_relaxed(words, pred_tags_seq)
    
    print(f"句子 {sentence_index} 分析:")
    print(f"文本: {''.join(words)}")
    print(f"字/词: {' '.join(words)}")
    # print(f"Gold标签: {' '.join(gold_tags)}")
    # print(f"Pred标签: {' '.join(pred_tags_seq)}")
    print(f"Gold实体: {_format_entities(gold_entities)}")
    print(f"Pred实体: {_format_entities(pred_entities)}")
    
    # 检查错误
    errors = []
    for i, (gold, pred) in enumerate(zip(gold_tags, pred_tags_seq)):
        if gold != pred:
            errors.append(f"位置{i}({words[i]}): {gold}->{pred}")
    
    if errors:
        print(f"标签错误: {', '.join(errors)}")
    else:
        print("✓ 无标签错误")

In [21]:

# 按实体类型统计错误
def analyze_entity_errors(results):
    """按实体类型分析错误"""
    if not results:
        return
        
    error_counter = results['error_counter']
    
    entity_errors = {
        'LOC': {'total': 0, 'errors': []},
        'PER': {'total': 0, 'errors': []},
        'ORG': {'total': 0, 'errors': []}
    }
    
    for error_type, count in error_counter.items():
        # 解析错误类型，如 "B-LOC->O"
        parts = error_type.split('->')
        if len(parts) == 2:
            gold_tag = parts[0]
            pred_tag = parts[1]
            
            # 检查是否涉及实体标签
            for entity_type in ['LOC', 'PER', 'ORG']:
                if entity_type in gold_tag:
                    entity_errors[entity_type]['total'] += count
                    entity_errors[entity_type]['errors'].append((error_type, count))
    
    print("\n按实体类型统计错误:")
    for entity_type, info in entity_errors.items():
        print(f"\n{entity_type}实体错误:")
        print(f"  总错误数: {info['total']}")
        for error_type, count in sorted(info['errors'], key=lambda x: x[1], reverse=True)[:5]:
            print(f"    {error_type}: {count}")

In [14]:
# 获取错误统计的前n个
def show_top_errors(results, top_n=10):
    """显示前N个最常见的错误类型"""
    error_counter = results['error_counter']
    print(f"\n前{top_n}个最常见的错误类型:")
    print("-" * 50)
    for i, (error_type, count) in enumerate(error_counter.most_common(top_n)):
        print(f"{i+1:2d}. {error_type}: {count} 次")

In [23]:
def analyze_sentences_by_error_severity(results, top_n=10):
    sentences = results['sentences']
    true_tags = results['true_tags']
    pred_tags = results['pred_tags']
    
    # 计算每个句子的错误统计
    sentence_errors = []
    
    for i, (words, gold_tags, pred_tags_seq) in enumerate(zip(sentences, true_tags, pred_tags)):
        # 计算错误数量
        error_count = sum(1 for gold, pred in zip(gold_tags, pred_tags_seq) if gold != pred)
        # 计算错误率
        error_rate = error_count / len(words) if len(words) > 0 else 0
        # 计算实体级别的错误
        gold_entities = _extract_entities_bmes_relaxed(words, gold_tags)
        pred_entities = _extract_entities_bmes_relaxed(words, pred_tags_seq)
        # 实体匹配度（简单的Jaccard相似度）
        gold_entity_set = set((e['text'], e['type']) for e in gold_entities)
        pred_entity_set = set((e['text'], e['type']) for e in pred_entities)
        if len(gold_entity_set) + len(pred_entity_set) > 0:
            entity_similarity = len(gold_entity_set & pred_entity_set) / len(gold_entity_set | pred_entity_set)
        else:
            entity_similarity = 1.0
        # 严重程度评分（错误率权重 + 实体错误权重）
        severity_score = error_rate * 0.6 + (1 - entity_similarity) * 0.4
        sentence_errors.append({
            'index': i,
            'sentence': ''.join(words),
            'words': words,
            'gold_tags': gold_tags,
            'pred_tags': pred_tags_seq,
            'error_count': error_count,
            'error_rate': error_rate,
            'gold_entities': gold_entities,
            'pred_entities': pred_entities,
            'entity_similarity': entity_similarity,
            'severity_score': severity_score
        })
    # 按严重程度排序
    sentence_errors.sort(key=lambda x: x['severity_score'], reverse=True)
    print(f"\n🔴 错误最严重的前{top_n}个句子:")
    print("=" * 80)
    for i, sent_data in enumerate(sentence_errors[:top_n]):
        print(f"\n📝 句子 #{i+1} (原始索引: {sent_data['index']})")
        print(f"   严重程度评分: {sent_data['severity_score']:.3f}")
        print(f"   错误数量: {sent_data['error_count']}/{len(sent_data['words'])}")
        print(f"   错误率: {sent_data['error_rate']:.1%}")
        print(f"   实体相似度: {sent_data['entity_similarity']:.1%}")
        print(f"   原文: {sent_data['sentence']}")
        # print(f"   字/词: {' '.join(sent_data['words'])}")   
        
        # 显示实体对比
        gold_entities_str = _format_entities(sent_data['gold_entities'])
        pred_entities_str = _format_entities(sent_data['pred_entities'])
        print(f"   Gold实体: {gold_entities_str}")
        print(f"   Pred实体: {pred_entities_str}")

In [39]:
def calculate_weighted_errors(results, top_n=20):
    """
    计算加权错误统计，考虑错误数量和标签分布
    results : dict
        评估结果
    top_n : int
        显示前N个加权错误类型
    """
    true_tags = results['true_tags']
    pred_tags = results['pred_tags']
    error_counter = results['error_counter']
    # 1. 统计所有标签的分布
    tag_distribution = Counter()
    for tags in true_tags:
        tag_distribution.update(tags)   
    total_tags = sum(tag_distribution.values())
    # 2. 计算每个标签的出现频率
    tag_frequency = {tag: count/total_tags for tag, count in tag_distribution.items()}
    # 3. 计算加权错误分数
    weighted_errors = {}
    for error_type, error_count in error_counter.items():
        gold_tag, pred_tag = error_type.split('->')
        # 基于金标频率的加权
        # 如果某个标签本身很少出现，但错误很多，说明模型对这个标签掌握不好
        gold_frequency = tag_frequency.get(gold_tag, 0)
        if gold_frequency > 0:
            # 错误率 = 错误数量 / 该标签总出现次数
            error_rate = error_count / tag_distribution[gold_tag]
            # 加权分数 = 错误率 * 错误数量（既考虑绝对数量也考虑相对错误率）
            weighted_score = error_rate * error_count
        else:
            weighted_score = 0
        weighted_errors[error_type] = {
            'count': error_count,
            'gold_frequency': gold_frequency,
            'error_rate': error_count / tag_distribution[gold_tag] if gold_tag in tag_distribution else 0,
            'weighted_score': weighted_score
        }
    # 4. 按加权分数排序
    sorted_errors = sorted(weighted_errors.items(), 
                          key=lambda x: x[1]['weighted_score'], 
                          reverse=True)
    
    print(f"\n🔍 加权错误分析 (前{top_n}个):")
    print(f"{'排名':<4} {'错误类型':<10} {'错误数量':<6} {'标签频率':<6} {'错误率':<4} {'加权分数':<4}")
    
    for i, (error_type, info) in enumerate(sorted_errors[:top_n]):
        print(f"{i+1:<4} {error_type:<15} {info['count']:<8} "
              f"{info['gold_frequency']:.4f}  {info['error_rate']:.4f}   {info['weighted_score']:.2f}")
    
    return weighted_errors, tag_distribution

def comprehensive_error_analysis(results, top_n=20):
    # 1.基础统计
    error_counter = results['error_counter']
    total_errors = sum(error_counter.values())
    print(f"总错误数: {total_errors}")
    print(f"错误类型数: {len(error_counter)}")
    # 2.加权错误分析
    weighted_errors, tag_distribution = calculate_weighted_errors(results, top_n)
    # 3.识别严重的问题
    print(f"\n🚨 严重的模型问题识别:")
    # 找出错误率超过50%的标签
    high_error_rate_tags = []
    for error_type, info in weighted_errors.items():
        if info['error_rate'] > 0.5 and info['count'] > 10:  # 错误率>50%且错误数量>10
            high_error_rate_tags.append((error_type, info['error_rate'], info['count']))
    
    high_error_rate_tags.sort(key=lambda x: x[1], reverse=True)
    if high_error_rate_tags:
        print("高错误率问题 (错误率 > 50%):")
        for error_type, error_rate, count in high_error_rate_tags[:5]:
            print(f"  {error_type}: {error_rate:.1%} ({count}次错误)")
    else:
        print("没有发现错误率超过50%的严重问题")
    return weighted_errors

In [40]:
if __name__ == "__main__":
    # 运行评估
    print("开始模型评估...")
    results = evaluate_model()
    if results:
        # 按错误数量排序
        #show_top_errors(results)
        # 按错误率排序
        weighted_results = comprehensive_error_analysis(results, top_n=10)
        # 按实体类型分析错误
        #analyze_entity_errors(results)
        # 按严重程度展示前n个句子
        analyze_sentences_by_error_severity(results, top_n=10)
        # 查看特定错误类型
        # interactive_error_analysis(results, 'B-PER->O')
        # 分析特定句子
        # compare_entities(results, 0)

开始模型评估...
读取数据...
加载模型: F:\GitHub\named_entity_recognition\ckpts\bilstm_crf.pkl
开始预测...

基础评估指标：
          标签    precision       recall     f1-score      support
     overall       0.8746       0.7941       0.8324        24143
         LOC       0.8441       0.8330       0.8385         7564
         ORG       0.8286       0.7874       0.8075         6181
         PER       0.9328       0.7698       0.8435        10398
总错误数: 6592
错误类型数: 64

🔍 加权错误分析 (前10个):
排名   错误类型       错误数量   标签频率   错误率  加权分数
1    E-PER->O        739      0.0137  0.2107   155.68
2    M-ORG->O        769      0.0175  0.1718   132.15
3    B-PER->O        630      0.0141  0.1756   110.65
4    M-PER->O        558      0.0129  0.1689   94.27
5    E-ORG->O        193      0.0033  0.2263   43.67
6    M-LOC->O        284      0.0080  0.1391   39.50
7    E-LOC->O        280      0.0108  0.1014   28.40
8    B-LOC->O        259      0.0108  0.0938   24.30
9    B-ORG->O        132      0.0033  0.1547   20.43
10   B-ORG->B-LOC  